In [ ]:
from sklearn import datasets
from torch import nn
import torch

In [ ]:
breast_cancer = datasets.load_breast_cancer()

breast_cancer_features_raw = breast_cancer.data
breast_cancer_labels_raw = breast_cancer.target

In [ ]:
def FNN(input_dim, hidden_dim, output_dim):
    pass

def BP():
    pass

In [ ]:
class FNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        result = self.linear_relu_stack(x)
        return result
    
def train(model, data, labels):
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
    loss_fn = nn.BCELoss()

    for t in range(100000):
        y_pred = model(data)
        #print(y_pred.view(-1))
        loss = loss_fn(y_pred, labels)

        #print(f"Epoch {t + 1}: {loss.item()}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def predict(model, data):
    model.eval()
    with torch.no_grad():
        y_pred = model(data)
        y_pred[y_pred < 0.5] = 0
        y_pred[y_pred >= 0.5] = 1
        return y_pred.view(-1)

def test(model, data, labels):
    model.eval()
    with torch.no_grad():
        y_pred = predict(model, data)
        correct = [1 for i in range(len(y_pred)) if y_pred[i] == labels[i]]
        accuracy = len(correct) / len(y_pred)
        print(f"Accuracy: {accuracy * 100}%")

In [ ]:
model = FNN().to('cpu')

data = torch.tensor(breast_cancer_features_raw[:285], dtype=torch.float32)
labels = torch.tensor(breast_cancer_labels_raw[:285], dtype=torch.float32)
labels = labels.view(-1, 1)

train(model, data, labels)

In [ ]:
test_data = torch.tensor(breast_cancer_features_raw[285:], dtype=torch.float32)
test_labels = torch.tensor(breast_cancer_labels_raw[285:], dtype=torch.float32).view(-1, 1)

test(model, test_data, test_labels)